In [1]:
if 'google.colab' in str(get_ipython()):
    import sys
    from google.colab import drive, output
    drive.mount('/content/drive')

    !pip install umap-learn
    !pip install umap-learn[plot]
    !pip install holoviews

    !pip install joypy

    output.clear()
    print("Running on colab")
    %load_ext autoreload
    %autoreload 1
    %cd '/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project'
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Dataloader')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Models')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Modules')
elif "zmqshell" in str(get_ipython()):
    print("Running on MIRP")
    root = "/home/mirp_ai/Documents/Daniel_Nieto/PhD/AutoEncoders_Ecoacoustics"
    root_path = "/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido"
else:
    import pathlib
    temp = pathlib.PosixPath
    pathlib.PosixPath = pathlib.WindowsPath
    print("Running local")

Running on MIRP


In [2]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import datetime
import gc
import pandas as pd
import joypy

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler as Normalizer

import torch
import torch.nn as nn

from torch.utils.data import DataLoader
from torch.utils.data import random_split

import torchvision.datasets as datasets

from Zamuro_DataLoader import SoundscapeData
from Models import ConvAE as AE
from AE_training_functions import TestModel, TrainModel
from AE_Clustering import AE_Clustering 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from datetime import timedelta

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import umap

import random
def _set_seed(seed: int):
    """
    Helper function for reproducible behavior to set the seed in ``random``, ``numpy``, ``torch`` and/or ``tf`` (if
    installed).
 
    Args:
        seed (:obj:`int`): The seed to set.
    """
    random.seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        # ^^ safe to call this function even if cuda is not available
_set_seed(1024)

/home/mirp_ai/anaconda3/envs/DANM/lib/python3.10/site-packages/torch/cuda/__init__.py:88: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /opt/conda/conda-bld/pytorch_1666643003845/work/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


# AE Model

In [3]:
model_type = "AE"
identifier = "batch_size_14_num_hiddens_64_"
day = 4
hour = 9
date_format = f"day_{day}_hour_{hour}"

model_name = f"{root}/Zamuro/temporal_zamuro/models/model_{model_type}_{identifier}_{date_format}_final.pth"
config = torch.load(f'{root}/Zamuro/temporal_zamuro/configs/config_{model_type}_{identifier}_{date_format}.pth', map_location=torch.device('cpu'))
model = AE(num_hiddens=config["num_hiddens"]).to(device)
model.load_state_dict(torch.load(f'{model_name}', map_location=torch.device('cpu')))

# y = torch.load(f"temporal_zamuro/Features/{folder}/AE_labels_{date_format}_No_rain_Audios_Jaguas.pth",  map_location=torch.device('cpu'))
X = torch.load(f"{root}/Zamuro/temporal_zamuro/Features/AE_features_Zamuro.pth",  map_location=torch.device('cpu'))
y_path = torch.load(f"{root}/Zamuro/temporal_zamuro/Features/AE_test_path_samples_Zamuro.pth",  map_location=torch.device('cpu'))
path_flat = [item for sublist in y_path for item in sublist]
path_flat = np.asarray(path_flat)
print(X.shape)

scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)
# Normalizer_ = Normalizer().fit(X)
# X_norm = Normalizer_.transform(X)
# PCA_ = PCA(n_components=60).fit(X_norm)
# X_PCA = PCA_.transform(X_norm)
# # X_TSNE = TSNE(n_components=60, learning_rate="auto", init='random', random_state=0).fit_transform(X_PCA)
# reducer = umap.UMAP(min_dist=0.9, n_components=60)
# X_UMAP = reducer.fit_transform(X_norm)
# X_batch = np.reshape(X_UMAP, (X_UMAP.shape[0]//5,5,X_UMAP.shape[1]))
# # X_UMAP_Norm = Normalizer().fit_transform(X_UMAP)

torch.Size([325320, 5184])


In [4]:
root_audios_Zamuro = f"{root}/Zamuro/Complementary_Files/Audios_Zamuro/zamuro_audios.csv"
root_recorders_Zamuro = f"{root}/Zamuro/Complementary_Files/zamuro_recorders.csv"
root_ai_Zamuro = f"{root}/Zamuro/AI_Caract/AI_Zamuro.csv"
root_vggish_Zamuro = f"{root}/Zamuro/vggish/Features_vggish"

audios = pd.read_csv(root_audios_Zamuro, index_col=0)
recorders = pd.read_csv(root_recorders_Zamuro, index_col=0)
ai = pd.read_csv(root_ai_Zamuro, index_col=0)
ai.dropna(inplace=True)
ai.set_index("file",inplace=True, drop=True)
ai.drop(columns=["Date"], inplace=True)
def combinar_nombre_ubicacion(row):
    return f"{row['field_number_PR']}_{row['Filename']}"

# Aplicando la función a cada fila del DataFrame para crear la nueva columna
audios['Filename_'] = audios.apply(combinar_nombre_ubicacion, axis=1)

audios.set_index("Filename_", inplace=True)

In [40]:
root_audios_Zamuro = f"{root}/Zamuro/Complementary_Files/Audios_Zamuro/zamuro_audios.csv"
root_recorders_Zamuro = f"{root}/Zamuro/Complementary_Files/zamuro_recorders.csv"
root_ai_Zamuro = f"{root}/Zamuro/AI_Caract/AI_Zamuro.csv"
root_vggish_Zamuro = f"{root}/Zamuro/vggish/Features_vggish"

audios = pd.read_csv(root_audios_Zamuro, index_col=0)
recorders = pd.read_csv(root_recorders_Zamuro, index_col=0)
ai = pd.read_csv(root_ai_Zamuro, index_col=0)
ai.dropna(inplace=True)
ai.set_index("file",inplace=True, drop=True)
ai.drop(columns=["Date"], inplace=True)
def combinar_nombre_ubicacion(row):
    return f"{row['field_number_PR']}_{row['Filename']}"

# Aplicando la función a cada fila del DataFrame para crear la nueva columna
audios['Filename_'] = audios.apply(combinar_nombre_ubicacion, axis=1)

audios.set_index("Filename_", inplace=True)


('RZUG11_20220906_214500.WAV_4',
 array([ 0.17501162,  0.11407401,  0.17391293, ..., -0.31956811,
        -0.22422653, -0.76522111]))

In [44]:
prueba = 133943
print(path_flat[prueba], X_scaled[prueba])

for i in range(len(X_3)):
    if (X_3[i] == X_scaled[prueba]).all():
        print(y_4[i])

RZUG11_20220906_214500.WAV_4 [ 0.17501162  0.11407401  0.17391293 ... -0.31956811 -0.22422653
 -0.76522111]
RZUG11_20220906_104500.WAV


In [56]:
X_vgg = torch.load(f"{root_vggish_Zamuro}/vggish_features.pth")

In [ ]:
len(X_vgg3), len(y_vggp), len(X_vgg2), len(y_vgg2)

### VGGish data

In [ ]:
y_ = np.asarray(y_path)
y_2 = y_[:,0]
for i in range(len(y_2)):
    y_2[i] = y_2[i][0:-2] 
y_3 = list(y_2)

X_vgg = torch.load(f"{root_vggish_Zamuro}/vggish_features.pth")
y_vgg = torch.load(f"{root_vggish_Zamuro}/vggish_filenames.pth")
cont = 0
vgg_corrupted = []
X_corrupted = []
for i in range(len(X_vgg)):
    if(X_vgg[i].shape!= torch.Size([62, 128])):
        vgg_corrupted.append(y_vgg[i])
        X_corrupted.append(i)
for i in range(len(vgg_corrupted)):
    y_vgg.remove(vgg_corrupted[i])        
X_vgg = np.delete(X_vgg, X_corrupted)

print(len(y_vgg), len(X_vgg))

y_vgg2 = [i.split('/')[6:8] for i in y_vgg]
X_vgg2 = []

print(len(y_vgg2))
for i in range(len(y_vgg2)):
    y_vgg2[i] = f"{y_vgg2[i][0]}_{y_vgg2[i][1]}"

X_vgg2 = X_vgg
X_vgg3 = []
remove = []
remove_idx = []
keep_idx = []
print(len(y_vgg2))
y_vggp = y_vgg2.copy()
print(len(y_vggp))


for i in range(len(y_vgg2)):
    #a[i] = X_vgg2[i]
    if y_vgg2[i] not in y_3:
        remove.append(y_vgg2[i])
        remove_idx.append(X_vgg2[i])
    else:
        keep_idx.append(i)
        
for i in range(len(remove)): 
    y_vgg2.remove(remove[i])
    
for i in keep_idx:
    X_vgg3.append(X_vgg2[i])
    
remove = []
keep_idx = []
X_2 = []
for i in range(len(y_3)):
    if y_3[i] not in y_vgg2:
        remove.append(y_vgg2[i])
    else:
        keep_idx.append(i)

X_1 = np.reshape(X_scaled, (X_scaled.shape[0]//5, 5, X_scaled.shape[1]))

for i in range(len(remove)): 
    y_3.remove(remove[i])
    
for i in keep_idx:
    X_2.append(X_1[i])
    
X_3 = np.asarray(X_2)
X_3 = np.reshape(X_3, (X_3.shape[0]*X_3.shape[1], X_3.shape[2]))
y_4 = np.repeat(y_3, 5)
        
print(len(y_4), len(X_3))
    
X_vgg4 = np.zeros([len(X_vgg3),62,128])
for i in range (len(X_vgg3)):
    X_vgg4[i] = X_vgg3[i].numpy()
    
X_vgg4 = X_vgg4.reshape(X_vgg4.shape[0], X_vgg4.shape[1]*X_vgg4.shape[2])
labels = []
for i in range(len(y_vgg2)):
    labels.append(audios.loc[y_vgg2[i], "cover"])
    
from sklearn.model_selection import train_test_split
X_train, X_test, labels_train, labels_test = train_test_split(X_vgg4, labels, test_size=0.2,random_state=0)

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
clf_rf = RandomForestClassifier(max_depth=32, random_state=0, n_jobs=-1)
clf_rf.fit(X_train, labels_train)
y_pred_rf = clf_rf.predict(X_test)
accuracy_vggish = metrics.accuracy_score(labels_test, y_pred_rf)
f1_score_vggish = metrics.f1_score(labels_test, y_pred_rf, average="macro")
recall_vggish = metrics.recall_score(labels_test, y_pred_rf, average="macro")
print("Accuracy:", accuracy_vggish)
print("f1:", f1_score_vggish)
print("recall", recall_vggish)

/home/mirp_ai/anaconda3/envs/DANM/lib/python3.10/site-packages/numpy/lib/function_base.py:5071: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = asarray(arr)
/home/mirp_ai/anaconda3/envs/DANM/lib/python3.10/site-packages/numpy/lib/function_base.py:5071: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asarray(arr)


71556 71556
71556
71556
71556


In [ ]:
Normalizer_ = Normalizer().fit(X_3)
X_norm = Normalizer_.transform(X_3)
PCA_ = PCA(n_components=60).fit(X_norm)
X_PCA = PCA_.transform(X_norm)
X_TSNE = TSNE(n_components=2, learning_rate="auto", init='random', random_state=0).fit_transform(X_norm)
reducer = umap.UMAP(min_dist=0.9, n_components=2)
X_UMAP = reducer.fit_transform(X_norm)
# X_batch = np.reshape(X_UMAP, (X_UMAP.shape[0]//5,5,X_UMAP.shape[1]))
# X_UMAP_Norm = Normalizer().fit_transform(X_UMAP)

### Acoustic Indices Data

In [ ]:
X_ai = []
remove = []
s=0
for i in range(len(y_3)):
    
    try:
        X_ai.append(ai.loc[y_3[i]])
    except:
        s+=1
        remove.append(y_3[i])
for i in range(len(remove)):
    y_3.remove(remove[i])
X_ai = np.asarray(X_ai)

labels_ai = []
# audios.set_index("Filename", inplace=True)
for i in range(len(y_3)):
    labels_ai.append(audios.loc[y_3[i], "cover"])
    
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_ai, labels_ai, test_size=0.2,random_state=0)

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

clf_rf = RandomForestClassifier(max_depth=32, random_state=0, n_jobs=-1)
clf_rf.fit(X_train, y_train)
y_pred_rf = clf_rf.predict(X_test)
accuracy_ai = metrics.accuracy_score(y_test, y_pred_rf)
f1_score_ai = metrics.f1_score(y_test, y_pred_rf, average="macro")
recall_ai = metrics.recall_score(y_test, y_pred_rf, average="macro")
print("Accuracy:", accuracy_ai)
print("f1:", f1_score_ai)
print("recall", recall_ai)

### Autoencoders Features and Labels using independent segments

In [ ]:
set(audios["cover"])

In [ ]:
set(audios["habitat"])

In [ ]:
from sklearn import svm
from sklearn.model_selection import train_test_split

# audios.set_index("Filename", inplace=True)
labels = []
for i in range(len(y_4)):
    labels.append(audios.loc[y_4[i], "cover"])

X_train, X_test, y_train, y_test = train_test_split(X_3, labels, test_size=0.2,random_state=0)

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
clf_rf = RandomForestClassifier(max_depth=32, random_state=0, n_jobs=-1)
clf_rf.fit(X_train, y_train)

y_pred_rf = clf_rf.predict(X_test)
accuracy_ae = metrics.accuracy_score(y_test, y_pred_rf)
f1_score_ae = metrics.f1_score(y_test, y_pred_rf, average="macro")
recall_ae = metrics.recall_score(y_test, y_pred_rf, average="macro")
print("Accuracy:", accuracy_ae)
print("f1:", f1_score_ae)
print("recall", recall_ae)


### Autoencoders Features and Labels using 5 segments of the audio

In [ ]:
X_c = X_3
y_c = y_4

X_batch = np.reshape(X_c, (X_c.shape[0]//5,5,X_c.shape[1]))
y_path = np.reshape(y_c, (y_c.shape[0]//5, 5))

In [ ]:
from sklearn import svm
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_batch, y_path, test_size=0.2,random_state=0)

X_train = X_train.reshape(X_train.shape[0]*X_train.shape[1], X_train.shape[2])
X_test = X_test.reshape(X_test.shape[0]*X_test.shape[1], X_test.shape[2])
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)
y_train = y_train.reshape(y_train.shape[0]*y_train.shape[1])
y_test = y_test.reshape(y_test.shape[0]*y_test.shape[1])

labels_train = []
for i in range(len(y_train)):
    labels_train.append(audios.loc[y_train[i], "cover"])
labels_test = []
for i in range(len(y_test)):
    labels_test.append(audios.loc[y_test[i], "cover"])

# Autoencoders Features and Labels using voting labels

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
clf_rf = RandomForestClassifier(max_depth=32, random_state=0, n_jobs=-1)
clf_rf.fit(X_train, labels_train)
y_pred_rf = clf_rf.predict(X_test)
accuracy = metrics.accuracy_score(labels_test, y_pred_rf)
f1_score = metrics.f1_score(labels_test, y_pred_rf, average="macro")
recall = metrics.recall_score(labels_test, y_pred_rf, average="macro")
print("Accuracy:", accuracy)
print("f1:", f1_score)
print("recall", recall)

In [ ]:
y_pred_rf2 = np.asarray(y_pred_rf)
y_pred_rf2 = np.reshape(y_pred_rf2,(y_pred_rf2.shape[0]//5,5))
y_test2 = np.asarray(labels_test)
y_test2 = np.reshape(y_test2,(y_test2.shape[0]//5,5))

def most_frequent(List):
    return max(set(List), key = List.count)

labels_test2 = []
labels_pred = []
for i in range(len(y_pred_rf2)):
    labels_pred.append(most_frequent(list(y_pred_rf2[i])))
    labels_test2.append(most_frequent(list(y_test2[i])))
accuracy = metrics.accuracy_score(labels_test2, labels_pred)
f1_score = metrics.f1_score(labels_test2, labels_pred, average="macro")
recall = metrics.recall_score(labels_test2, labels_pred, average="macro")

print("Accuracy:", accuracy)
print("f1:", f1_score)
print("recall", recall)

In [ ]:
f1_scores = [f1_score_ai, f1_score_ae, f1_score_vggish] # [ai, AE_norm, AE_PCA, AE_UMAP, VGGISH, VGGISH_PCA, VGGISH_UMAP]
accuracies = [accuracy_score_ai, accuracy_score_ae, accuracy_score_vggish]
recalls = [recall_score_ai, recall_score_ae, recall_score_vggish]
